# Create Project

In [1]:
# Install tmdbsimple (only need to run once)
!pip install tmdbsimple

In [2]:
 !pip install tqdm


## Load libraries

In [15]:
# import libraries
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import StrMethodFormatter
import matplotlib.ticker as ticker

#import 
import os, time


# API tools and json
import json
import tmdbsimple as tmdb
from tqdm.notebook import trange, tqdm
from tqdm.notebook import tqdm_notebook 



# settings
pd.options.display.float_format = '{:,.2f}'.format

## Functions

In [6]:
def get_movie_with_rating(movie_id):
    #get the movie and release dates
    movie = tmdb.Movies(movie_id)
    ##contruct output dict
    info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            info['certification'] = c['certification']
    return info

In [7]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename, 'r+') as file:
        file_data = json.load(file)
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        file.seek(0)
        json.dump(file_data, file)

## Load data

In [9]:
# open the json
with open("C:\\Users\\Elizabeth Spreng\\.secret\\tmbd_api.json", 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [12]:
# load tmbd simple
tmdb.API_KEY =  login['api-key']

## Test functions
- This test will check that the functions are running correctly for 2 movie ids: tt0848228 ("The Avengers") and tt0332280 ("The Notebook").

In [13]:
# using function
avengers = get_movie_with_rating('tt0848228')
notebook = get_movie_with_rating('tt0332280')


In [14]:
# confirming results
for i in [avengers, notebook]:
    print(i['title'], i['release_date'], i['certification'])

The Avengers 2012-04-25 PG-13
The Notebook 2004-06-25 PG-13


## Create Folder

In [16]:
FOLDER = "API_Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

[]

# Loop for API

In [18]:
# define  years for the loop
YEARS_TO_GET = [2000,2001]

In [19]:
# define errors
errors = [ ]

## API Loop

In [20]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)

    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()

    # saving movie ids to list
    movie_ids = df['tconst'].copy()
    movie_ids
    
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    previous_df

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
            
    # save to file
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1450 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1570 [00:00<?, ?it/s]

In [21]:
# checking the number of errors
print(f"- Total errors: {len(errors)}")

# demonstrating that the errors are related to imdb ids not found in the tmdb
count = 0
not_404 = []
for i in errors:
    if '404 Client Error' in str(i[1]):
        count += 1
    else:
        not_404.append(i)
print(f'- 404 Client Errors: {count}')

# printing any errors that are not 404 if they exist
if not_404: 
    print(not_404)

- Total errors: 442
- 404 Client Errors: 442
